# Fase 1 de proyecto MLOps

## Setup
### Importar librerías

In [34]:
#from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

### Importar dataset
Existen 3 formas de dataset:  
1. Desde dataset en el repositorio original
2. Desde datset en el repositorio modificado
3. Desde internet (se utiliza ucimlrepo, a veces da problemas con proxy)
4. Dataset online (bypass ucimlrepo)

In [9]:
## Seleccionar modo de importación:
## 1. Desde repositorio, original
## 2. Desde repositorio, modificado
## 3. Desde internet

dataset_selection=1

def load_dataset(dataset_selection):
    match dataset_selection:
        case 1:
            df=pd.read_csv('/workspaces/bike_sharing_mlops/data/raw/bike_sharing_original.csv')
            return df
        case 2:
            df=pd.read_csv('/workspaces/bike_sharing_mlops/data/raw/bike_sharing_modified.csv')
            return df
        case 3:
            # fetch dataset 
            bike_sharing = fetch_ucirepo(id=275) 
            
            # data (as pandas dataframes) 
            X = bike_sharing.data.features 
            y = bike_sharing.data.targets 
            
            # metadata 
            print(bike_sharing.metadata) 
            
            # variable information 
            print(bike_sharing.variables)
        case 4:
            url='https://archive.ics.uci.edu/static/public/275/data.csv'
            df=pd.read_csv(url)
            return df
        case _:
            return "Invalid option"  # Default case

df_orig=load_dataset(1)
df_mod=load_dataset(2)
df_online=load_dataset(4)

df_online.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## EDA

### Cargando y limpiando dataset
Se programan variables de intención global para analisis de datos por categorías o numéricos

In [51]:
## Selecionar dataset para usar "df" en el resto del código
df=df_online
df.set_index('instant')

df.info()

a=df.count()
b=df.count(numeric_only=True)

print(f"El número de variables numéricas es {b.size}")
print(f"El número de variables no númericas (categóricas) es: {a.size-b.size}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB
El número de variables numéricas es 16
El número 

In [61]:
def clean_dataset(df):
    df_clean=df.dropna()
    df_clean=df_clean.drop_duplicates()
    return df_clean

df2=clean_dataset(df)
a=df.dtypes
missing=100*df.isna().sum()/df.count()
missing
df2.set_index('instant')

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,,,,,,,,,,,,,,,,
1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90


In [62]:
descriptives=df2.describe()
#Media, mediana, min, max, desviación estandar y cuartiles
descriptives 

simmetry=df2.skew(numeric_only=True)
outlierly=df2.kurt(numeric_only=True)
print(simmetry)
print(outlierly)
print(descriptives)

instant       0.000000
season       -0.005416
yr           -0.010243
mnth         -0.009253
hr           -0.010680
holiday       5.638538
weekday      -0.002998
workingday   -0.785258
weathersit    1.228047
temp         -0.006021
atemp        -0.090429
hum          -0.111287
windspeed     0.574905
casual        2.499237
registered    1.557904
cnt           1.277412
dtype: float64
instant       -1.200000
season        -1.334251
yr            -2.000125
mnth          -1.201878
hr            -1.198021
holiday       29.796540
weekday       -1.255997
workingday    -1.383529
weathersit     0.350151
temp          -0.941844
atemp         -0.845412
hum           -0.826117
windspeed      0.590820
casual         7.571002
registered     2.750018
cnt            1.417203
dtype: float64
          instant        season            yr          mnth            hr  \
count  17379.0000  17379.000000  17379.000000  17379.000000  17379.000000   
mean    8690.0000      2.501640      0.502561      6.537775     